In [78]:
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0
    
def count_diff(list_1, list_2):
    count = 0
    for x in range(0, len(list_1)):
        if list_1[x] != list_2[x]:
            count += 1
    return count

def build_phi(points, clusters, centers):
    #BUILD PHI TO GET WEIGHTS
    phi = []
    for m in range(0, len(points)):
        point = points[m]
        row = []
        for n in range(0, clusters):
            center = centers[n]
            temp = numpy.exp(gam*-1 * numpy.linalg.norm(numpy.subtract(point, center))* numpy.linalg.norm(numpy.subtract(point, center)))
            row.append(temp)
        phi.append(row)
    return phi

In [83]:
import numpy
from random import uniform
from sklearn import svm, cluster
import math

N = 100
gam = 1.5
clusters = 9
iterations = 1000


E_i = 0

for i in range(0, iterations):
    data = [[uniform(-1, 1), uniform(-1, 1)] for x in range(0, N)]
    data_out = [[uniform(-1, 1), uniform(-1, 1)] for x in range(0, N)]

    def f(point):
        return sign(point[1] - point[0] + 0.25 * numpy.sin(numpy.pi*point[0]))

    class_data = [f(point) for point in data]
    class_data_out = [f(point) for point in data_out]

    """clf = svm.SVC(kernel="rbf", C=math.inf, gamma=gam, coef0 = 1)
    clf.fit(data, class_data)  """

    Kmeans = cluster.KMeans(n_clusters=clusters, init='random').fit(data)

    for i in range(0,clusters):
        if (i not in Kmeans.labels_):
            iterations += 1
            continue

    phi = numpy.array(build_phi(data, clusters, Kmeans.cluster_centers_))
    class_data = numpy.array(class_data)

    w = numpy.matmul(numpy.matmul(numpy.linalg.pinv(numpy.matmul(phi.T, phi)), phi.T), class_data.T)

    #KMEANS OUT PREDICTION
    phi_out = numpy.array(build_phi(data_out, clusters, Kmeans.cluster_centers_))

    y_out = numpy.matmul(phi_out, w.T)
    y_out = [sign(x) for x in y_out]

    y_in = numpy.matmul(phi, w.T)
    y_in = [sign(x) for x in y_in]

    E_out_Kmeans = count_diff(y_out, class_data_out)/len(y_out)
    E_in_Kmeans = count_diff(y_in, class_data)/len(y_in)
    if E_in_Kmeans == 0.0:
        E_i += 1/iterations

print(E_i)

0.014000000000000005
